In [ ]:
%load_ext autoreload
%autoreload 2
%aimport
%matplotlib inline

In [ ]:
import os
import sys
nb_dir = os.path.dirname(os.path.split(os.getcwd())[0])
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [ ]:
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from turicreate import SFrame, load_sframe
from pathlib import Path
import turicreate.aggregate as agg
import numpy as np

In [ ]:
import json
import os
import matplotlib.pyplot as plt
import pandas as pd
import math
import glob
import ntpath
from tqdm import tqdm
from itertools import chain
import seaborn as sns
from  matplotlib.ticker import FuncFormatter
from matplotlib.backends.backend_pdf import PdfPages


## Utility Functions

In [ ]:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i + n]

def get_d(sf_corr):
    for data in sf_corr.groupby("Disease"):
        if len(data[1]) >5:
            yield data[1].sort_values("Year")

### To PDF

In [ ]:

plt.figure(figsize=(20, 10))

sns.set(style="ticks")
def create_gird(df, col, hue,x,y,sharey=True, legend=False):

    # Initialize a grid of plots with an Axes for each walk
    grid = sns.FacetGrid(df, col=col, hue=hue, palette=sns.color_palette("hls", 4),sharey=sharey,
                         col_wrap=3, height=4.5)
    plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))

    # Draw a horizontal line to show the starting point
    grid.map(plt.axhline, y=0, ls=":", c=".5")

    # Draw a line plot to show the trajectory of each random walk
    grid.map(plt.plot, x, y)
    if legend:
        grid.add_legend()



    # Adjust the arrangement of the plots
    grid.fig.tight_layout(w_pad=1)
    return grid

## Analysis

In [ ]:
spothlight = ["SARS","MERS Coronavirus", "Avian Influenza","Ebola", "Influenza", "HIV/AIDS","Hepatitis B","Hepatitis C", "Swine Flu"]

In [ ]:
diseases_id = load_sframe("Data/diseases_id.csv")
disease_names = pd.read_csv("Data/disease_names.csv")

In [ ]:
diseases_mag = load_sframe("Data/mag/diseases_med_mag.sframe")

In [ ]:
diseases_mag = diseases_mag[diseases_mag["Ref Number"]>5]

#### Diseases Researchers

In [ ]:
diseases_mag_authors = diseases_mag.stack("Authors List Sorted",new_column_name="author")

In [ ]:
diseases_mag_authors = diseases_mag_authors[diseases_mag_authors["Year"]>2001]

In [ ]:
diseases_researchers = diseases_mag_authors.groupby("disease",  { "author_count": agg.COUNT_DISTINCT("author")}).sort('author_count',ascending=False)

In [ ]:
diseases_researchers = diseases_researchers.filter_by(spothlight,"disease")

In [ ]:
diseases_researchers

New Authros Over Time (Not inclided in the papaer):

In [ ]:
disease_authors = dict(zip(spothlight, [set()]*9))
res = {"Year":[],"Disease":[],"Value":[],"Type":[]}
for year in tqdm(range(2002,2019)):
    for d in spothlight:
        author_list = diseases_mag[(diseases_mag["disease"]==d)&(diseases_mag["Year"]==year)&(diseases_mag["Authors List Sorted"]!=None)]["Authors List Sorted"]
        author_list = set(chain(*author_list))
        res["Year"].append(year)
        res["Disease"].append(d)
        res["Type"].append("New Authors")
        res["Value"].append(len(author_list-disease_authors[d]))
        res["Year"].append(year)
        res["Disease"].append(d)
        res["Type"].append("Distinct Authors")
        res["Value"].append(len(author_list))
        disease_authors[d] = disease_authors[d]|author_list

In [ ]:
new_authors = pd.DataFrame(res)

In [ ]:
new_authors = new_authors.sort_values(["Disease", "Year"])

In [ ]:


plt.rc('text', usetex=False)
plt.figure(figsize=(20, 10))
create_gird(new_authors,"Disease","Type","Year", "Value",False, legend=True)


In [ ]:
autor_disease =  diseases_mag[(diseases_mag["Year"]>2001)&(diseases_mag["Ref Number"] >5)][["PaperId","disease","Authors List Sorted"]].stack("Authors List Sorted",new_column_name="Author")

In [ ]:
autor_disease = autor_disease.filter_by(spothlight, "disease")

In [ ]:
autor_disease_count = autor_disease.groupby(["disease", "Author"], {"Count":agg.COUNT()})

Average Number of Papers for each author on a specific disease:

In [ ]:
autor_disease_count.to_dataframe()[["disease","Count"]].groupby("disease").mean()


Average Number of Papers for authors with more than one paper on a specific disease:

In [ ]:
mutli_papaers = autor_disease_count[autor_disease_count["Count"]>1].to_dataframe()[["disease","Count"]].groupby("disease").mean()
mutli_papaers

In [ ]:
mutli_papaers.loc["HIV/AIDS"] - mutli_papaers.loc["SARS"]

In [ ]:
from ScienceDynamics.datasets.microsoft_academic_graph import MicrosoftAcademicGraph
mag = MicrosoftAcademicGraph()

In [ ]:
diseases_mag_authors = diseases_mag_authors.filter_by(spothlight,"disease")

In [ ]:
diseases_authors = diseases_mag.stack("Authors List Sorted",new_column_name="author")

In [ ]:
authors = set(diseases_mag_authors["author"])

In [ ]:
# This will take sometime
extended_papers = mag.extended_papers.stack("Authors List Sorted",new_column_name="author")

In [ ]:
authors_papers = extended_papers.filter_by(list(authors),"author")

In [ ]:
author_first_paper = authors_papers.groupby("author",{"Year":agg.MIN("Year")})

In [ ]:
diseases_mag_authors = diseases_mag_authors.join(author_first_paper,on="author")

In [ ]:
diseases_mag_authors["seniority when published"] = diseases_mag_authors["Year"] - diseases_mag_authors["Year.1"]

General Seniority:

In [ ]:
diseases_mag_authors.groupby("disease", {"seniority":agg.AVG("seniority when published"), "median":agg.QUANTILE("seniority when published", [0.5])})

In [ ]:
diseases_mag["FirstAuthor"] = diseases_mag["Authors List Sorted"].apply(lambda x: x[0])

In [ ]:
diseases_mag["LastAuthor"] = diseases_mag["Authors List Sorted"].apply(lambda x: x[-1])

In [ ]:
diseases_mag = diseases_mag.filter_by(spothlight,"disease")

In [ ]:
diseases_mag = diseases_mag.join(author_first_paper,on={"FirstAuthor": "author"},alter_name={"Year":"FirstYear"})

In [ ]:
diseases_mag = diseases_mag.join(author_first_paper,on={"LastAuthor": "author"},alter_name={"Year":"LastYear"})

In [ ]:
diseases_mag["First author seniority when published"] = diseases_mag["Year"] - diseases_mag["FirstYear"]

In [ ]:
diseases_mag["Last author seniority when published"] = diseases_mag["Year"] - diseases_mag["LastYear"]

In [ ]:
diseases_mag.groupby("disease", {"seniority first":agg.AVG("First author seniority when published"), "median first":agg.QUANTILE("First author seniority when published", [0.5]), "seniority last":agg.AVG("Last author seniority when published"), "median last":agg.QUANTILE("Last author seniority when published", [0.5])})

In [ ]:
res = SFrame()
for d in spothlight:
    print(d)
    sf = diseases_mag[diseases_mag["disease"]==d]
    first = sf["seniority when published"].value_counts()[:10]
    res = res.add_columns(first.rename({"value":f"first {d}", "count":f"first count {d}"}))
    last = sf["seniority when published last"].value_counts()[:10]
    res = res.add_columns(last.rename({"value":f"last {d}", "count":f"last count {d}"}))
#     print("-----First------")
#     print(sf["seniority when published"].value_counts())
#     print("-----Last------")
#     print(sf["seniority when published last"].value_counts())

In [ ]:
res

In [ ]:
disease_authors = dict(zip(spothlight, [set()]*9))
res = {"Year":[],"Disease":[],"Value":[],"Type":[]}
for year in tqdm(range(2002,2019)):
    for d in spothlight:
        author_list = diseases_mag[(diseases_mag["Disease"]==d)&(diseases_mag["Year"]==year)&(diseases_mag["Authors List Sorted"]!=None)]["Authors List Sorted"]
        author_list = set(chain(*author_list))
        res["Year"].append(year)
        res["Disease"].append(d)
        res["Type"].append("New Authors")
        res["Value"].append(len(author_list-disease_authors[d]))
        res["Year"].append(year)
        res["Disease"].append(d)
        res["Type"].append("Distinct Authors")
        res["Value"].append(len(author_list))
        disease_authors[d] = disease_authors[d]|author_list

In [ ]:
new_authors = pd.DataFrame(res)

In [ ]:
new_authors = new_authors.sort_values(["Disease", "Year"])